In [1]:
!pip install contractions


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd

In [3]:
import contractions               #Won't -> will not
import re                         #Regex
import nltk                       #Tokenizar texto
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from tqdm import tqdm

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

stopwords_en = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
tqdm.pandas()

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>
[nltk_data] Error loading punkt_tab: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/1252/Procesamiento de Lenguaje Natural/Archivos/Articles.csv', encoding = "latin-1")

In [ ]:
df.head()

In [ ]:
df['NewsType'].unique()

In [ ]:
embeddings = {}

with open('/content/drive/MyDrive/1252/Procesamiento de Lenguaje Natural/Archivos/glove.6B/glove.6B.300d.txt', encoding='utf-8') as f:
  for line in f:
    values = line.split()
    word = values[0]
    vectors = np.asarray(values[1:])
    embeddings[word] = vectors

In [ ]:
def vectorize(texto):
  vector_size = 300 #Dependiento del archivo de glove -> glove.6B.300d.txt

  #Todo a minusculas
  texto = texto.lower()

  #Contracciones
  texto = contractions.fix(texto)

  #Regex para quitar todo lo que no sea letra
  texto = re.sub(r'[^a-z\s]','', texto)

  #Quitar stop words
  texto = word_tokenize(texto)

  #Eliminar stopwords
  texto = [token for token in texto if token not in stopwords_en]

  #Lemmatizar
  texto = [lemmatizer.lemmatize(word) for word in texto]

  #Generar vector (sumar linealmente)
  vector = np.zeros(vector_size)

  for word in texto:
    if word in embeddings:
      vector += embeddings[word].astype(float)

  vector = vector.reshape(1, -1)[0] #Una fila, columans necesarias

  return vector

In [ ]:
vectorize('This is a test')

In [ ]:
df['vector'] = df['Article'].progress_apply(vectorize)

In [ ]:
def buscar_documentos(documento_original, n):
  data = {
      'Article':[],
      'sims':[]
  }

  buscar = vectorize(documento_original)
  A = buscar
  for vector, headline in zip(df['vector'], df['Article']):

    B = vector

    try:
      resultado = np.dot(A,B) / (np.linalg.norm(A) * np.linalg.norm(B))
    except:
      resultado = 0

    data['Article'].append(headline)
    data['sims'].append(resultado)

  final = pd.DataFrame(data).sort_values(by='sims', ascending=False).head(n)
  return final

In [ ]:
buscar_documentos('Asian market plummet', 15)

In [ ]:
print(buscar_documentos('Asian market plummet', 15)['Article'].iloc[0])

In [ ]:
print(buscar_documentos('Roger Federer', 15)['Article'].iloc[0])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

In [ ]:
X = df['vector']
X = np.concatenate(X, axis=0).reshape(-1, 300)
y = df['NewsType']

X_test, X_train, y_test, y_train = train_test_split(X,
                                                    y,
                                                    train_size = 0.7,
                                                    random_state=101)

In [ ]:
modelo = RandomForestClassifier()
modelo.fit(X_train, y_train)

In [ ]:
y_pred = modelo.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='.2f')
print(classification_report(y_test,y_pred))